# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
df = pd.read_csv('csv_outputs/city_weather.csv')
df = df.drop(columns=['Unnamed: 0'])
df.head()

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed (mph),Country,Date
0,Qinhuangdao,39.9317,119.5883,69.96,25,4,15.12,CN,1620357497
1,Bredasdorp,-34.5322,20.0403,60.80,88,83,11.50,ZA,1620357418
2,Nikolskoye,59.7035,30.7861,42.80,87,90,4.47,RU,1620357385
3,Mataura,-46.1927,168.8643,57.99,79,73,0.13,NZ,1620357406
4,Punta Arenas,-53.1500,-70.9167,42.80,87,75,9.22,CL,1620357269


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
#Location dataframe
locations = df[['Latitude', 'Longitude']]

#humidity dataframe (vector?)
humidity = df['Humidity'].astype(float)

#create map/heat map
figure = gmaps.figure()
heatmap = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False,max_intensity=20,point_radius=1)
figure.add_layer(heatmap)

figure


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#Max temperature lower than 80 degrees but >= 40.
#Wind speed less than 15 mph.
#Cloudiness <= than 50.
#Humidity <=70

#new df
ideal_spot = df[['City','Max Temperature', 'Wind Speed (mph)', 'Cloudiness', 'Humidity','Latitude','Longitude']]

#define ideal numbers
temp= (ideal_spot['Max Temperature'] < 75) & (ideal_spot['Max Temperature'] >= 40)
wind= (ideal_spot['Wind Speed (mph)'] <= 10)
humid= (ideal_spot['Humidity'] <= 65)
clouds= (ideal_spot['Cloudiness'] <= 50)

spot_filtered = ideal_spot[temp & wind & humid & clouds]
spot_filtered

,City,Max Temperature,Wind Speed (mph),Cloudiness,Humidity,Latitude,Longitude
33,Shahreza,64.08,2.71,17,30,32.0339,51.8794
57,Bemidji,41.00,5.75,1,42,47.4736,-94.8803
66,Abhā,60.80,2.66,0,44,18.2164,42.5053
67,Khorixas,67.05,5.30,19,56,-20.3667,14.9667
69,Weyburn,42.80,9.22,0,26,49.6668,-103.8511
151,Yellowknife,51.80,8.05,20,35,62.4560,-114.3525
192,Xining,57.54,3.24,0,19,36.6167,101.7667
244,Los Andes,66.99,4.61,0,50,-32.8337,-70.5983
298,Mount Gambier,68.00,9.22,40,64,-37.8333,140.7667
305,Yangi Marg`ilon,69.80,4.52,40,60,40.4272,71.7189


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = pd.DataFrame(spot_filtered, columns=['City', 'Latitude', 'Longitude'])
hotel_df['Hotel Name'] =''
hotel_df = hotel_df.reset_index()
hotel_df

,index,City,Latitude,Longitude,Hotel Name
0,33,Shahreza,32.0339,51.8794,
1,57,Bemidji,47.4736,-94.8803,
2,66,Abhā,18.2164,42.5053,
3,67,Khorixas,-20.3667,14.9667,
4,69,Weyburn,49.6668,-103.8511,
5,151,Yellowknife,62.4560,-114.3525,
6,192,Xining,36.6167,101.7667,
7,244,Los Andes,-32.8337,-70.5983,
8,298,Mount Gambier,-37.8333,140.7667,
9,305,Yangi Marg`ilon,40.4272,71.7189,


In [16]:
hotel_names= []

#set parameters
radius = 5000
keyword = 'hotel'

#loop through cities to get lat and lng
for index, city in hotel_df.iterrows():
    lat = hotel_df.loc[index,'Latitude']
    lng = hotel_df.loc[index,'Longitude']
    #url
    url = (f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?key={g_key}&radius={radius}&location={lat},{lng}&keyword={keyword}')
    
    try:
        response = (requests.get(url)).json()
        hotel_names.append(response['results'][0]['name'])
        
    except:
        hotel_names.append("")
        
          
                                 



In [17]:
hotel_names

['هتل باغ اناری',
 'DoubleTree by Hilton Hotel Bemidji',
 'Shafa Abha Hotel',
 '',
 'Microtel Inn & Suites by Wyndham Weyburn',
 'The Explorer Hotel',
 'Sofitel Xining',
 'Hotel Los Andes',
 'Commodore on the Park',
 'Diyor hotel',
 'Cuesta Real Hotel',
 '',
 'Tongcheng Hotel',
 'Pueblo del Soul B&B',
 'Holiday Inn Express & Suites Durant, an IHG Hotel',
 'Raoum Inn Hotel',
 'Grouse Mountain Lodge',
 '',
 'Best Western Premier Ivy Inn & Suites']

In [23]:
hotel_df["Hotel Name"]= hotel_names
hotel_df.replace('', np.nan, inplace=True)
hotel_df = hotel_df.dropna(how= 'any')
hotel_df.head()

,index,City,Latitude,Longitude,Hotel Name
0,33,Shahreza,32.0339,51.8794,هتل باغ اناری
1,57,Bemidji,47.4736,-94.8803,DoubleTree by Hilton Hotel Bemidji
2,66,Abhā,18.2164,42.5053,Shafa Abha Hotel
4,69,Weyburn,49.6668,-103.8511,Microtel Inn & Suites by Wyndham Weyburn
5,151,Yellowknife,62.4560,-114.3525,The Explorer Hotel


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [26]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
figure.add_layer(markers)


# Display figure
figure

Figure(layout=FigureLayout(height='420px'))